In [5]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id

GPU_IDS = '0,1'
current_gpu = choose_gpu_by_id(GPU_IDS)
print(current_gpu)
from src.utils.notebook_imports import *
from pyforest import *

# jupyter magic config
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import own modules and utils
from src.utils.utils_io import Console_and_file_logger, ensure_dir, init_config
import src.utils.my_metrics as metr
from src.models.train_model import train_kfolded
# train kfolded data, start each fold manually to improve speed
from src.data.dataset import get_kfolded_data

# define aditional imports
import tensorflow as tf
import pandas as pd
import glob
import datetime
import json
import keras

# define experiment name for report, model and log paths + filenames
EXPERIMENT = '2D/new_kfold/tetra'
now = datetime.datetime.now()

# image params
DIM = [224, 224]
IMG_WIDTH = 224
IMG_HEIGHT = 224
# Greyscale images
IMG_CHANNELS = 1
# RV = 1 = Y[...,0] 
# Myo = 2 = Y[...,1] 
# LV = 3 = Y[...,2]
MASK_VALUES = [0, 1, 2, 3]  
MASK_CLASSES = len(MASK_VALUES)
AUGMENT = False
SHUFFLE = True
AUGMENT_GRID = True
RESAMPLE = False
SPACING_X = 1.00
SPACING_Y = 1.00
SCALER = 'MinMax'


AUGMENT_IMAGE_PARAMS = dict(  # samplewise_center=True,
                # samplewise_std_normalization=True,
                #horizontal_flip=True,
                #vertical_flip=True,
                #rotation_range=30,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.1,
                # fill_mode='nearest'
            )
AUGMENT_MASK_PARAMS = dict(  # samplewise_center=True,
                # samplewise_std_normalization=True,
                #horizontal_flip=True,
                #vertical_flip=True,
                #rotation_range=30,
                width_shift_range=0.1,
                height_shift_range=0.1,
                zoom_range=0.1,
                fill_mode='nearest',
                # dtype=np.bool
            )
# path params
DATASET = 'tetra'  # 'acdc' # or 'tetra'
DATA_ROOT = 'data/raw/tetra/2D/'
TRAIN_PATH = 'data/raw/tetra/2D/train/'
VAL_PATH = 'data/raw/tetra/2D/val/'
TEST_PATH = 'data/raw/tetra/2D/test/'

MODEL_PATH = os.path.join(os.path.join('models', EXPERIMENT), str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join(os.path.join('reports/tensorboard_logs', EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join(os.path.join('reports/configs/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join(os.path.join('reports/history/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))

# training params
ARCHITECTURE = '2D'
seed = 42
BATCHSIZE =  24 # 32, 64, 16, 1
INITIAL_EPOCH = 0
EPOCHS = 150
FOLDS = 4
EPOCHS_BETWEEN_CHECKPOINTS = 5
MONITOR_FUNCTION = 'val_dice_coef_labels'
MONITOR_MODE = 'max'

# Network params
OPTIMIZER = 'Adam'  # adam, sgd, softmax # https://keras.io/optimizers/
ACTIVATION = 'elu'  # 'elu' --> works well with binary_crossentropy and bce_dice_loss, relu does not work, it clips negative values, bse does return negative values
LEARNING_RATE = 0.001
DECAY = 0.0
EPSILON = 1e-08
DROPOUT_L1_L2 = 0.3 # best with 0.4 and other 0.5
DROPOUT_L3_L4 = 0.4
DROPOUT_L5 = 0.5
BATCH_NORMALISATION = True


LOSS_FUNCTION = metr.bce_dice_jac_loss  # 'binary_crossentropy'--> 91 IOU, bce_dice_loss --> 94 IOU
#LOSS_FUNCTION = metr.weighted_cce_dice_coef([1,5,5,3])
#LOSS_FUNCTION = keras.losses.categorical_crossentropy
#LOSS_FUNCTION = metr.weighted_categorical_crossentropy([1,5,10,3])
#LOSS_FUNCTION = keras.losses.binary_crossentropy
metrics = [
    metr.dice_coef_labels,
    #metr.dice_coef_background,
    metr.dice_coef_myo,
    metr.dice_coef_lv,
    metr.dice_coef_rv,
]

# make sure logging directories are available
Console_and_file_logger(EXPERIMENT, logging.INFO)


# Define a config for param injection and save it for usage during evaluation, save all upper key,value pairs from global namespace
config = dict(((key, value) for key, value in locals().items()
               if key.isupper() and key not in ['HTML', 'K']))

# loss and metrics cant be seriealized
write_config = init_config(config, False)


# define a Tensorflow config
#tf_config = tf.ConfigProto()
#tf_config=tf.ConfigProto(
#      allow_soft_placement=True, log_device_placement=True)
#tf_config.gpu_options.allow_growth = True
#tf_session = tf.Session(config=tf_config)
#tf.keras.backend.set_session(tf_session)

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7945901818760172153
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 771506148398656707
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 410517504
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 6686838217789978721
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 372703232
locality {
  bus_id: 1
  links {
    link {
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 14053330287961213263
physical_device_desc: "device: 1, name: TITAN RTX, pci bus id: 0000:02:00.0, compute capability: 7

Using TensorFlow backend.
2019-10-01 09:29:18,893 INFO -------------------- Start --------------------
2019-10-01 09:29:18,893 INFO Working directory: /mnt/data/git/cardio.
2019-10-01 09:29:18,894 INFO Log file: ./logs/2D/new_kfold/tetra.log


In [ ]:
# call train
train_kfolded(config, metrics)

2019-07-24 11:03:45,771 INFO config saved:
 {
    "ACTIVATION": "elu",
    "ARCHITECTURE": "2D",
    "AUGMENT": false,
    "AUGMENT_IMAGE_PARAMS": {
        "height_shift_range": 0.1,
        "width_shift_range": 0.1,
        "zoom_range": 0.1
    },
    "AUGMENT_MASK_PARAMS": {
        "fill_mode": "nearest",
        "height_shift_range": 0.1,
        "width_shift_range": 0.1,
        "zoom_range": 0.1
    },
    "BATCHSIZE": 16,
    "BATCH_NORMALISATION": true,
    "CONFIG_PATH": "reports/configs/2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\\2019-07-24_11_03_fold0",
    "DATASET": "tetra",
    "DATA_ROOT": "data/raw/tetra/2D/",
    "DECAY": 0.0,
    "DIM": [
        224,
        224
    ],
    "DROPOUT_L1_L2": 0.4,
    "DROPOUT_L3_L4": 0.5,
    "DROPOUT_L5": 0.5,
    "EPOCHS": 150,
    "EPOCHS_BETWEEN_CHECKPOINTS": 5,
    "EPSILON": 1e-08,
    "EXPERIMENT": "2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4",
    "FOLDS": 4,
    "GPU_DEVICE": "/device:GPU

Epoch 1/150
655/655 [==============================] - 306s 467ms/step - loss: -0.6538 - bce_dice_jac_loss: -0.6538 - dice_coef: 0.9551 - dice_coef_labels: 0.6964 - dice_coef_background: 0.9756 - dice_coef_myo: 0.6104 - dice_coef_lv: 0.7608 - dice_coef_rv: 0.6964 - val_loss: -0.7093 - val_bce_dice_jac_loss: -0.7093 - val_dice_coef: 0.9761 - val_dice_coef_labels: 0.7401 - val_dice_coef_background: 0.9887 - val_dice_coef_myo: 0.6222 - val_dice_coef_lv: 0.8318 - val_dice_coef_rv: 0.7519


2019-07-24 11:09:37,209 INFO Saved model to disk: models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-24_11_03_fold0



Epoch 00001: dice_coef_labels improved from -inf to 0.69637, saving model to models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-24_11_03_fold0\checkpoint.h5
Epoch 2/150
655/655 [==============================] - 289s 442ms/step - loss: -0.8010 - bce_dice_jac_loss: -0.8010 - dice_coef: 0.9778 - dice_coef_labels: 0.8216 - dice_coef_background: 0.9866 - dice_coef_myo: 0.7533 - dice_coef_lv: 0.8925 - dice_coef_rv: 0.8117 - val_loss: -0.7631 - val_bce_dice_jac_loss: -0.7631 - val_dice_coef: 0.9787 - val_dice_coef_labels: 0.7874 - val_dice_coef_background: 0.9887 - val_dice_coef_myo: 0.7072 - val_dice_coef_lv: 0.8412 - val_dice_coef_rv: 0.7959

Epoch 00002: dice_coef_labels improved from 0.69637 to 0.82158, saving model to models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-24_11_03_fold0\checkpoint.h5
Epoch 3/150
655/655 [==============================] - 279s 426ms/step - loss: -0.8241 - bce_dice_jac_loss: -0.8241 - dice_coef: 0.9788 - di

C:\Users\s.koehler\AppData\Local\conda\conda\envs\sv\lib\site-packages\keras\callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.218400). Check your callbacks.
  % delta_t_median)


...

655/655 [==============================] - 834s 1s/step - loss: -0.9112 - bce_dice_jac_loss: -0.9112 - dice_coef: 0.1466 - dice_coef_labels: 0.9191 - dice_coef_background: 0.0608 - dice_coef_myo: 0.8559 - dice_coef_lv: 0.9556 - dice_coef_rv: 0.9269 - val_loss: -0.8462 - val_bce_dice_jac_loss: -0.8462 - val_dice_coef: 0.0795 - val_dice_coef_labels: 0.8602 - val_dice_coef_background: 0.0019 - val_dice_coef_myo: 0.8045 - val_dice_coef_lv: 0.9160 - val_dice_coef_rv: 0.8536

Epoch 00149: dice_coef_labels did not improve from 0.91955
Epoch 150/150
655/655 [==============================] - 841s 1s/step - loss: -0.9112 - bce_dice_jac_loss: -0.9112 - dice_coef: 0.1483 - dice_coef_labels: 0.9191 - dice_coef_background: 0.0629 - dice_coef_myo: 0.8552 - dice_coef_lv: 0.9552 - dice_coef_rv: 0.9272 - val_loss: -0.8448 - val_bce_dice_jac_loss: -0.8448 - val_dice_coef: 0.1201 - val_dice_coef_labels: 0.8592 - val_dice_coef_background: 0.0480 - val_dice_coef_myo: 0.8077 - val_dice_coef_lv: 0.9155 - val

2019-07-25 06:21:04,276 INFO Fold: 0 finished - saving scores.
2019-07-25 06:21:04,354 INFO {'val_loss': [-0.7092917736603006, -0.7631045248476379, -0.8051925649022842, -0.8146548893954187, -0.8130885450829305, -0.8261322213395295, -0.815876760557628, -0.8304421858937217, -0.8292250037193298, -0.8294824094515745, -0.830447255228667, -0.8273212463331864, -0.826922457314393, -0.8357745978329748, -0.8336560263762025, -0.8320109959140487, -0.8396434297476114, -0.8337597173425649, -0.8308438380737476, -0.8338523926756307, -0.8369427617355313, -0.8335545776670824, -0.8304952810698026, -0.8253102198309963, -0.8398436227721484, -0.8337753671701713, -0.8330727118013151, -0.8376261583892753, -0.8333606883014799, -0.8195655680023501, -0.8461035131873571, -0.841670927980021, -0.8361241261520728, -0.8358582939802264, -0.8282801233599538, -0.8270143714186322, -0.8351505048606428, -0.8344997282520004, -0.8378405891726369, -0.8417200713414248, -0.8340600467048953, -0.8337429109710215, -0.8354331840314

Epoch 1/150
5187/5187 [==============================] - 1360s 262ms/step - loss: -0.6344 - bce_dice_jac_loss: -0.6344 - dice_coef: 0.9446 - dice_coef_labels: 0.6675 - dice_coef_background: 0.9575 - dice_coef_myo: 0.5804 - dice_coef_lv: 0.7137 - dice_coef_rv: 0.6658 - val_loss: -0.5654 - val_bce_dice_jac_loss: -0.5654 - val_dice_coef: 0.9538 - val_dice_coef_labels: 0.6075 - val_dice_coef_background: 0.9730 - val_dice_coef_myo: 0.5838 - val_dice_coef_lv: 0.7448 - val_dice_coef_rv: 0.5664


2019-07-25 06:44:38,054 INFO Saved model to disk: models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-25_06_21_fold1



Epoch 00001: dice_coef_labels improved from -inf to 0.66747, saving model to models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-25_06_21_fold1\checkpoint.h5
Epoch 2/150
5187/5187 [==============================] - 1351s 261ms/step - loss: -0.7212 - bce_dice_jac_loss: -0.7212 - dice_coef: 0.8778 - dice_coef_labels: 0.7435 - dice_coef_background: 0.8819 - dice_coef_myo: 0.6617 - dice_coef_lv: 0.7997 - dice_coef_rv: 0.7324 - val_loss: -0.4683 - val_bce_dice_jac_loss: -0.4683 - val_dice_coef: 0.7164 - val_dice_coef_labels: 0.5374 - val_dice_coef_background: 0.7272 - val_dice_coef_myo: 0.4904 - val_dice_coef_lv: 0.5260 - val_dice_coef_rv: 0.6101

Epoch 00002: dice_coef_labels improved from 0.66747 to 0.74349, saving model to models\2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4\2019-07-25_06_21_fold1\checkpoint.h5
Epoch 3/150
5187/5187 [==============================] - 1370s 264ms/step - loss: -0.7411 - bce_dice_jac_loss: -0.7411 - dice_coef: 0.738

# Train crossvalidation on ACDC or Fallot data

In [2]:
# define current fold
from src.data.dataset import create_acdc_dataframe_for_cv
fold = 3

# ACDC DATA
#kfold_data_dir = 'reports/kfolds_data/2D/unet_acdc_spacing1_batch32_drop04_batchnorm_minmax_kfold/'
# create new split

# df_folds = create_acdc_dataframe_for_cv()

# save split
#ensure_dir(kfold_data_dir)
#df_folds.to_csv(os.path.join(kfold_data_dir, 'df_kfold_acdc.csv'))

# load split
#df_folds = pd.read_csv(os.path.join(kfold_data_dir, 'df_kfold_acdc.csv'))

########################
# FALLOT DATA
kfold_data_dir = 'reports/kfolds_data/2D/unet_tetra_spacing1_batch32_drop04_batchnorm_minmax_kfold4'

# create new split
#df_folds = get_kfolded_data(kfolds=config['FOLDS'], path_to_data=config['DATA_ROOT'])

# save split
#df_folds.to_csv(os.path.join(kfold_data_dir, 'df_kfold.csv'))

# load split
df_folds = pd.read_csv(os.path.join(kfold_data_dir, 'df_kfold.csv'))

df_fold = df_folds[df_folds['fold'] == fold]
train_kfolded(config, metrics, df_fold)

2019-08-30 13:06:59,335 INFO config saved:
 {
    "ACTIVATION": "elu",
    "ARCHITECTURE": "2D",
    "AUGMENT": false,
    "AUGMENT_GRID": true,
    "AUGMENT_IMAGE_PARAMS": {
        "height_shift_range": 0.1,
        "width_shift_range": 0.1,
        "zoom_range": 0.1
    },
    "AUGMENT_MASK_PARAMS": {
        "fill_mode": "nearest",
        "height_shift_range": 0.1,
        "width_shift_range": 0.1,
        "zoom_range": 0.1
    },
    "BATCHSIZE": 24,
    "BATCH_NORMALISATION": true,
    "CONFIG_PATH": "reports/configs/2D/new_kfold/tetra\\2019-08-30_13_06_fold3",
    "DATASET": "tetra",
    "DATA_ROOT": "data/raw/tetra/2D/",
    "DECAY": 0.0,
    "DIM": [
        224,
        224
    ],
    "DROPOUT_L1_L2": 0.3,
    "DROPOUT_L3_L4": 0.4,
    "DROPOUT_L5": 0.5,
    "EPOCHS": 150,
    "EPOCHS_BETWEEN_CHECKPOINTS": 5,
    "EPSILON": 1e-08,
    "EXPERIMENT": "2D/new_kfold/tetra",
    "FOLDS": 4,
    "GPU_DEVICE": "/device:GPU:0",
    "GPU_ID": "0",
    "HISTORY_PATH": "reports/history

Epoch 1/150
434/434 [==============================] - 133s 307ms/step - loss: -0.1031 - dice_coef_labels: 0.2906 - dice_coef_myo: 0.2092 - dice_coef_lv: 0.3046 - dice_coef_rv: 0.3473 - val_loss: -0.3548 - val_dice_coef_labels: 0.4720 - val_dice_coef_myo: 0.3589 - val_dice_coef_lv: 0.5039 - val_dice_coef_rv: 0.4967


2019-08-30 13:09:56,354 INFO Saved model to disk: models\2D/new_kfold/tetra\2019-08-30_13_06_fold3



Epoch 00001: val_dice_coef_labels improved from -inf to 0.47201, saving model to models\2D/new_kfold/tetra\2019-08-30_13_06_fold3\checkpoint.h5
Epoch 2/150
434/434 [==============================] - 124s 285ms/step - loss: -0.7797 - dice_coef_labels: 0.7974 - dice_coef_myo: 0.7088 - dice_coef_lv: 0.8658 - dice_coef_rv: 0.7993 - val_loss: -0.7170 - val_dice_coef_labels: 0.7442 - val_dice_coef_myo: 0.6570 - val_dice_coef_lv: 0.8371 - val_dice_coef_rv: 0.7396

Epoch 00002: val_dice_coef_labels improved from 0.47201 to 0.74423, saving model to models\2D/new_kfold/tetra\2019-08-30_13_06_fold3\checkpoint.h5
Epoch 3/150
434/434 [==============================] - 119s 273ms/step - loss: -0.8189 - dice_coef_labels: 0.8347 - dice_coef_myo: 0.7609 - dice_coef_lv: 0.8998 - dice_coef_rv: 0.8312 - val_loss: -0.8282 - val_dice_coef_labels: 0.8434 - val_dice_coef_myo: 0.7889 - val_dice_coef_lv: 0.9097 - val_dice_coef_rv: 0.8338

Epoch 00003: val_dice_coef_labels improved from 0.74423 to 0.84340, savi

2019-08-30 13:38:47,405 INFO Fold: 3 finished - saving scores.
2019-08-30 13:38:47,405 INFO Training fold 3 done in 1908.070 s


Epoch 00015: early stopping


2019-08-30 13:38:47,688 INFO All folds are finished in 1908.354 sec
